In [2]:
import spacy

nlp = spacy.load('en_core_web_md')

In [25]:
chat_text = ('Hi my name is John Larsen, I am Danish, and my bank account 12345678 with sort code 20-17-09. \
                My VISA is 4444333322221111 how do I get access? \
                I think I have £325 on my account when I checked on 10-05-2011 \
                I have 2 accounts with you. \
                You can reach me on phone 07726 000 001 or johnglarsen@gmail.com. \
                Ohh and my address is 4 Hull Way, St Neots, Cambridgeshire')
chat_test_doc = nlp(chat_text)
for ent in chat_test_doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_, spacy.explain(ent.label_))

John Larsen 14 25 PERSON People, including fictional
Danish 32 38 NORP Nationalities or religious or political groups
20-17-09 84 92 CARDINAL Numerals that do not fall under another type
VISA 113 117 ORG Companies, agencies, institutions, etc.
4444333322221111 121 137 DATE Absolute or relative dates or periods
325 191 194 MONEY Monetary values, including unit
10-05-2011 227 237 DATE Absolute or relative dates or periods
2 261 262 CARDINAL Numerals that do not fall under another type
4 402 403 CARDINAL Numerals that do not fall under another type
Hull Way 404 412 LOC Non-GPE locations, mountain ranges, bodies of water
St Neots 414 422 PERSON People, including fictional
Cambridgeshire 424 438 GPE Countries, cities, states


In [27]:
from spacy import displacy

displacy.serve(chat_test_doc, style='ent')

E:\Anaconda\envs\ML-NLP-GPU\lib\runpy.py:193: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  "__main__", mod_spec)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


### Let's create a Custom Named Entity 